In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set()

In [2]:
import common
import importer

In [3]:
output, stats = importer.generate_output_and_stats(2018, "input\\2018.csv", 6)

In [6]:
stats["Tampa Bay Buccaneers"]

{'allowed': [40, 21, 30, 48, 34, 23],
 'points': [48, 27, 27, 10, 29, 26],
 'wins': [1, 1, 0, 0, 0, 1],
 'yards': [529, 436, 455, 311, 512, 456]}

In [7]:
stats["Cincinnati Bengals"]

{'allowed': [23, 23, 31, 36, 17, 28, 45],
 'points': [34, 34, 21, 37, 27, 21, 10],
 'wins': [1, 1, 0, 1, 1, 0, 0],
 'yards': [330, 373, 396, 407, 332, 275, 239]}

In [ ]:
data[data["home_pct"] == data["away_pct"]]["home_win"].describe()

In [ ]:
data[data["home_pct"] == 1]["home_win"].mean()

In [ ]:
data[data["home_pct"] > data["away_pct"]]["home_win"].describe()

In [ ]:
data[data["away_pct"] > data["home_pct"]]["home_win"].describe()

In [ ]:
data[data["away"] == "New Orleans Saints"]["home_win"].describe()

In [ ]:
#sns.pairplot(data, hue='home_win')

from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_true=y, y_pred=y_model)

sns.heatmap(mat.T, annot=True, fmt='d')

In [ ]:
data.head()

In [ ]:
X.head()

In [ ]:
MLPClassifier?